<a href="https://colab.research.google.com/github/amfei/Causal_Inference/blob/main/Causal_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Causal inference** is the process of determining whether one variable (often called the "treatment" or "cause") **directly affects another variable** (the "outcome" or "effect"). This is different from **correlation**, which simply measures whether two variables **move together**, without implying that one causes the other.

Example: The Effect of Studying on Exam Scores
Imagine you're a student trying to figure out if studying more hours leads to better exam scores. Here's how you might approach this question using causal inference:

Treatment (Cause): Number of hours spent studying.
Outcome (Effect): Exam score.
Confounders: Other variables that might affect both studying and exam scores (e.g., previous knowledge, hours of sleep, access to study materials).





# There are various methods for causal inference, including **randomized controlled trials (RCTs)**, **instrumental variables (IV), propensity score matching**, and more recently, methods using g**raphical models and machine learning.**

## I'll demonstrate a simple example of causal inference using the DoWhy library, which is designed for causal analysis. DoWhy provides a straightforward way to specify causal models and perform causal inference.


## **Propensity Score:** The probability of a unit (e.g., a customer) receiving the treatment given their observed characteristics.

## Problem: **Estimating the Effect of a** Loyalty Program **on** Insurance Claims
### Suppose an insurance company wants to estimate the effect of enrolling customers in a loyalty program on the number of insurance claims they file. Since customers are not randomly assigned to the loyalty program, there may be **confounding variables** that influence both the enrollment and the outcome (e.g., age, income, past claims history).

Steps for Propensity Score Application

**1. Data Preparation:** Create a dataset with treatment, outcome, and covariates.

**2. Propensity Score Estimation:** Estimate the probability of treatment assignment based on observed covariates.

**3. Matching/Weighting/Stratification:** Use propensity scores to balance the treatment and control groups.

**4. Outcome Analysis:** Compare the outcomes between the balanced groups to estimate the treatment effect.



In [ ]:
import numpy as np
import pandas as pd

# Seed for reproducibility
np.random.seed(42)

# Generate synthetic data
num_samples = 1000
data = {
    'age': np.random.randint(18, 70, size=num_samples),
    'income': np.random.normal(50000, 10000, size=num_samples),
    'past_claims': np.random.randint(0, 5, size=num_samples),
    'loyalty_program': np.random.binomial(1, 0.3, size=num_samples),  # 30% in loyalty program
}

df = pd.DataFrame(data)

# Introduce a causal relationship
df['num_claims'] = df['loyalty_program'] * -0.5 + df['past_claims'] * 0.3 + np.random.normal(0, 1, num_samples)
df['num_claims'] = df['num_claims'].apply(lambda x: max(x, 0))  # Ensure non-negative claims

# Display the first few rows of the dataset
print(df.head())


# **Propensity Score Estimation**
Estimate the propensity scores using logistic regression:


In [ ]:

from sklearn.linear_model import LogisticRegression

# Define features and treatment
X = df[['age', 'income', 'past_claims']]
y = df['loyalty_program']

# Fit logistic regression to estimate propensity scores
logistic = LogisticRegression()
logistic.fit(X, y)

# Predict propensity scores
df['propensity_score'] = logistic.predict_proba(X)[:, 1]

# Display the first few rows with propensity scores
print(df.head())

## **Matching:** Treated units (enrolled in the loyalty program) are matched with control units (not enrolled) based on their propensity scores using nearest neighbor matching.

## Matching is a method to pair each treated unit with one or more control units that have similar propensity scores. This aims to balance the distribution of observed covariates between the treated and control groups, reducing the bias due to confounding variables.

## Nearest Neighbor Matching: Each treated unit is matched with the closest control unit(s) based on the propensity score. In this example, we use 1-to-1 nearest neighbor matching.

### In observational studies, like our insurance example, the treatment (enrollment in a loyalty program) is not randomly assigned. This can lead to biased estimates of the treatment effect because the treated and untreated groups might differ in ways that also affect the outcome (number of claims). Matching helps to make these groups comparable.

### How do we do Matching?
We use propensity scores, which are the estimated probabilities that a customer will be in the loyalty program based on their characteristics (age, income, past claims). We then match each treated customer with one or more control customers who have similar propensity scores.

### Steps in Matching for our example:

**Estimate Propensity Scores:** Use logistic regression to calculate the probability of each customer being in the loyalty program based on their age, income, and past claims.

**Find Matches:** For each customer in the loyalty program (treated), find a customer not in the program (control) with a similar propensity score. This is like finding "twins" based on the probability of receiving the treatment.

**Create Matched Dataset:** Combine the treated customers and their matched controls into a new dataset that will be used for analysis.



In [ ]:
from sklearn.neighbors import NearestNeighbors

# Separate treated and control groups
treated = df[df['loyalty_program'] == 1]
control = df[df['loyalty_program'] == 0]

# Fit nearest neighbors model
nn = NearestNeighbors(n_neighbors=1)
nn.fit(control[['propensity_score']])

# Find nearest neighbors in the control group for each treated unit
distances, indices = nn.kneighbors(treated[['propensity_score']])

# Get matched control units
matched_controls = control.iloc[indices.flatten()]

# Combine treated and matched control units
matched_df = pd.concat([treated, matched_controls])

# Display the first few rows of the matched dataset
print(matched_df.head())


In [ ]:
# Calculate average treatment effect
ate = matched_df[matched_df['loyalty_program'] == 1]['num_claims'].mean() - \
      matched_df[matched_df['loyalty_program'] == 0]['num_claims'].mean()

print(f'Average Treatment Effect (ATE): {ate}')


## To solve the insurance loyalty claim problem using the **DoWhy** library, we can follow these steps:

Data Preparation: Create synthetic data with baseline features and introduce a causal relationship.

Causal Model Specification: Define the causal model including treatment, outcome, and confounders.

Identify the Causal Effect: Use DoWhy to identify the causal effect.

Estimate the Causal Effect: Estimate the causal effect using an appropriate method.

Refute the Estimate: Perform robustness checks to ensure the validity of the causal effect

In [ ]:
import dowhy
from dowhy import CausalModel

# Define the causal model
model = CausalModel(
    data=df,
    treatment='loyalty_program',
    outcome='num_claims',
    common_causes=['age', 'income', 'past_claims']
)

# View the model
model.view_model()


In [ ]:
identified_estimand = model.identify_effect()
print(identified_estimand)


In [ ]:
estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.linear_regression"
)

print(estimate)


# **Refutation**: We perform robustness checks to ensure the causal effect is not due to random chance or unobserved confounders.

In [ ]:
# Refute the causal estimate
refutation = model.refute_estimate(
    identified_estimand,
    estimate,
    method_name="placebo_treatment_refuter"
)

print(refutation)